In [ ]:
#Importing the required packages
import torch as tr
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import math
import time
import sys
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
#Defining some of the functions used
def awgn_channel(x):
    return x + np.sqrt(sigma2)*tr.randn(M*stacks,channel_uses).to(Device)
def normalization(x): # E[|x|^2] = 1
    return x / tr.sqrt((2*(x**2)).mean())
def save():
        tr.save({
         'model_state_dict' : encoder.state_dict(), 
         'optimizer_state_dict': optimizer.state_dict(),
         'loss': loss_history,
         'constellations': Constellations,
         'SNR': EsNo_dB,
         'epochs' : epochs,
         'stacks' : stacks,
         'learning_rate': learning_rate,
         'Device': Device,
         'time': time.time()-start_time},'./Data/MI/' + str(channel_uses) + 'D/' + str(M) + '/MI_' + Estimation_type_save + '_' + str(channel_uses) + 'D_' + str(M) + '_' + str(EsNo_dB) + 'dB_'+ str(learning_rate)+'lr')

        sio.savemat('./Data/MI/' + str(channel_uses) + 'D/' + str(M) + '/MI_' + Estimation_type_save + '_' + str(channel_uses) + 'D_' + str(M) + '_' + str(EsNo_dB) + 'dB_'+ str(learning_rate)+'lr', {
         'model_state_dict' : encoder.state_dict(), 
         'optimizer_state_dict': optimizer.state_dict(),
         'loss': loss_history,
         'constellations': Constellations,
         'SNR': EsNo_dB,
         'epochs' : epochs,
         'stacks' : stacks,
         'learning_rate': learning_rate,
         'Device': Device,
         'time': time.time()-start_time})


In [ ]:
#The input parameters of the optimization

M = 1024 #The cardinality size of the constellation
stacks = 100 #The amount of noise samples per symbol for MC optimization
channel_uses = 2 #The dimensionality of the constellation
learning_rate = 0.01 #Learning Rate
EsNo_dB = 26 # The SNR
epochs = 200 #The amount of iterations
Estimation_type = 'GH' #GH or MC
Device = 'cpu' #Determines the device which the optimization is done on, 'cpu' for cpu and 'cuda:0', 'cuda:1' etc. for GPU

In [1]:
#Defining some of the parameters of the optimization

m = np.log2(M) #The amount of bits per symbol
EsNo_r = 10**(EsNo_dB/10)
sigma2 = 1/(channel_uses*EsNo_r) # noise variance per channel use
GH =  sio.loadmat('GaussHermite_J_10.mat')#Loading in the Gauss-Hermite points
X_eye = tr.eye(M).to(Device) # The input to our neural network
X_tilde = X_eye.repeat(stacks,1).to(Device)
if Estimation_type == 'MC':
    Estimation_type_save = Estimation_type + '_' + str(stacks) + 'Stacks'
else:
    Estimation_type_save = Estimation_type

NameError: name 'np' is not defined

In [ ]:
#=====================================================#
# build the computation graph
#=====================================================#
def MI(X_tilde):
    X = normalization(encoder(X_tilde)) # minibatch_size x channel_uses
    Y = awgn_channel(X)                 # minibatch_size x channel_uses
    #print(X[0:M:1,:])
    # compute posterior distribution
    XX = tr.reshape(X[0:M:,:], shape=[M, 1, channel_uses]).to(Device)
    YY = tr.reshape(Y, shape=[1, M*stacks, channel_uses]).to(Device)

    num = tr.exp(-tr.sum((Y-X)**2,1)/(2*sigma2)) # f_{Y|X}
    den = (tr.exp(-tr.sum((YY-XX)**2,2)/(2*sigma2))).mean() # f_{Y}

    posterior = num/den

    epsilon = 0.000001 # to avoid log(x) = -Inf
    loss = -(tr.log(posterior + epsilon)/np.log(2)).mean()
    return loss

def MI_GH(X_tilde):
    X = normalization(encoder(X_tilde)) 
    GH_xi = tr.tensor(GH['xi'], dtype  = tr.float32).to(Device)#Load in the Gauss-Hermite points
    GH_alpha = tr.tensor(GH['alpha'], dtype = tr.float32).to(Device)#Load in the Gauss-Hermite weigths
    
    Dmat = tr.zeros(M,M,channel_uses).to(Device)
    Dmat[:,:,0] = X[:,0].unsqueeze(1) -(X[:,0].unsqueeze(1)).t() #Calculate the distances between constellation points
    Dmat[:,:,1] = X[:,1].unsqueeze(1) -(X[:,1].unsqueeze(1)).t()

    Es = (X[:,0]**2 + X[:,1]**2).mean() #Calculate the signal energy
    EsN0lin = 10**(EsNo_dB/10)  #Turn the SNR value from dB to a linear value
    SigmaZ2 = (Es/(EsN0lin)) #Calculate the noise variance
    sum_0 = 0 #Initialize the sum 

    for l1 in range(10): #Dimension 1
        for l2 in range(10): #Dimension 2
             num = tr.exp(-((Dmat[:,:,0]**2 + Dmat[:,:,1]**2) + 2*tr.sqrt(SigmaZ2)*(GH_xi[l1]*Dmat[:,:,0] - GH_xi[l2]*Dmat[:,:,1]))/SigmaZ2)
             sum_0 = GH_alpha[l1]*GH_alpha[l2]*tr.log(tr.sum(num,1))/tr.log(tr.tensor(2, dtype = tr.float32))  + sum_0
    sum_0 = tr.sum(sum_0)
    MI = m-1/M/math.pi*sum_0
    loss = -MI
    return loss

def MI_GH_4D(X_tilde):
    X = normalization(encoder(X_tilde)) 
    GH_xi = tr.tensor(GH['xi'], dtype  = tr.float32).to(Device)#Load in the Gauss-Hermite points
    GH_alpha = tr.tensor(GH['alpha'], dtype = tr.float32).to(Device)#Load in the Gauss-Hermite weigths
    
    Dmat = tr.zeros(M,M,channel_uses).to(Device)
    Dmat[:,:,0] = X[:,0].unsqueeze(1) -(X[:,0].unsqueeze(1)).t() #Calculate the distances between constellation points
    Dmat[:,:,1] = X[:,1].unsqueeze(1) -(X[:,1].unsqueeze(1)).t()
    Dmat[:,:,2] = X[:,2].unsqueeze(1) -(X[:,2].unsqueeze(1)).t()
    Dmat[:,:,3] = X[:,3].unsqueeze(1) -(X[:,3].unsqueeze(1)).t()
    Dmatnorm = Dmat[:,:,0]**2 + Dmat[:,:,1]**2+ Dmat[:,:,2]**2 + Dmat[:,:,3]**2
    Es = (X[:,0]**2 + X[:,1]**2 + X[:,2]**2 + X[:,3]**2).mean() #Calculate the signal energy
    EsN0lin = 10**(EsNo_dB/10)  #Turn the SNR value from dB to a linear value
    SigmaZ2 = (Es/(EsN0lin)) #Calculate the noise variance
    sum_0 = 0 #Initialize the sum 
    for l1 in range(10): #Dimension 1
        for l2 in range(10): #Dimension 2
            for l3 in range(10):
                for l4 in range(10):
                     num = tr.exp(-(Dmatnorm + np.sqrt(2)*tr.sqrt(SigmaZ2)*(GH_xi[l1]*Dmat[:,:,0] - GH_xi[l2]*Dmat[:,:,1] + GH_xi[l3]*Dmat[:,:,2] - GH_xi[l4]*Dmat[:,:,3]))/(0.5*SigmaZ2))
                     sum_0 = GH_alpha[l1]*GH_alpha[l2]*GH_alpha[l3]*GH_alpha[l4]*tr.log(tr.sum(num,1))/tr.log(tr.tensor(2, dtype = tr.float32))  + sum_0
    sum_0 = tr.sum(sum_0)
    MI = m-1/M/(math.pi**2)*sum_0
    loss = -MI
    return loss

In [ ]:
#Training the model
start_time = time.time()
encoder = tr.nn.Sequential()
encoder.add_module('last', tr.nn.Linear(M,channel_uses,bias = False))
encoder.to(Device)
optimizer = tr.optim.Adam(encoder.parameters(), learning_rate)
loss_history = np.zeros(epochs) # For saving the losses
Constellations = np.zeros((M,channel_uses,epochs)) #For saving the constellations
for i in range(1, epochs+1):
    if Estimation_type == 'GH':
        if channel_uses ==2:
            loss = MI_GH(X_eye)
        else:
            loss = MI_GH_4D(X_eye)
    else:
        loss = MI(X_tilde)
    loss_history[i-1] = loss
    Constellations[:,:,i-1] = normalization(encoder(X_eye)).data.numpy()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i%100 == 0 or i == 1:
        print('iter ', i, ' loss', loss, 'time', time.time()- start_time)
    save()

In [ ]:
#Plotting the final constellation
Constellation = normalization(encoder(X_tilde)).data.numpy() 
if channel_uses == 4:
    fig = plt.figure()
    ax = fig.add_subplot(131)
    ax.scatter(Constellation[:,0],Constellation[:,1], marker = '.')
    ax.set_aspect(1)
    ax2 = fig.add_subplot(132)
    ax2.scatter(Constellation[:,2],Constellation[:,3], marker = '.')
    ax2.set_aspect(1)
    ax3 = fig.add_subplot(133)
    ax3.plot(loss_history)
else:
    fig = plt.figure()
    ax = fig.add_subplot(121)
    ax.scatter(Constellation[:,0],Constellation[:,1], marker = '.')
    ax.set_aspect(1)
    ax2 = fig.add_subplot(122)
    ax2.plot(loss_history)

In [ ]:
#Creating an animation of the evolution of the constellation, only works for 2D constellations
plt.rcParams['animation.embed_limit'] = 2**128
def init():
    line[0].set_data([],[])
    line[1].set_data([],[])
    return(line,)

def animate(i):
    x = Constellations[:,0,i]
    y = Constellations[:,1,i]
    line[0].set_data(x,y)
    line[1].set_data(np.linspace(0,i,i),loss_history[0:i])
    return(line,)


fig = plt.figure()
ax = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)
ax.set_xlim((-1.8,1.8))
ax.set_ylim((-1.8,1.8))
ax2.set_xlim((-1,epochs))
ax2.set_ylim((-max(loss_history)-0.2,-min(loss_history)+0.2))
ax.set_aspect('equal')
ax2.set_xlabel('iterations')
ax2.set_ylabel('loss')
line1, = ax.plot([],[],'.',lw=2,)
line2, = ax2.plot([],[],lw=2,)
line1.set_color('b')
line2.set_color('b')
line = [line1, line2]
anim = animation.FuncAnimation(fig,animate,init_func = init, frames = epochs, interval = 10, blit = False)
anim.save('./Data/MI/' + str(channel_uses) + 'D/' + str(M) + '/MI_' + Estimation_type + '_' + str(channel_uses) + 'D_' + str(M) + '_' + str(EsNo_dB) + 'dB_'+ str(learning_rate)+'lr_animation.mp4') 
HTML(anim.to_jshtml())